# Ayudantía 06 - EDD201

**autores:** Santiago y Paula


Esperando que estén disfrutado un feliz 18 les damos la bienvenida a la ayudantía EDD201 de IIC2233. Se recomienda revisar antes el enunciado y plantearse el ejercicio, para que puedan usar esta solución para resolver sus dudas.

---

## Solución Propuesta

La solución que se propone en esta ayudantía, es la creación de dos clases que permitirán implementar el contactTrie y sus respectivos métodos. La primera corresponde a la clase `Node`, cuyo objetivo es modelar cada nodo del árbol y sus principales funciones, para poder dar así la estructura al árbol. Y la segunda ya es la clase `ContactTrie` que representa al árbol de contactos que comienza desde la raíz principal. La idea es que la primera clase ayude a construir al árbol, y permita su funcionamiento "por detrás" mientras que la clase `ContactTrie` sea la interfaz de la implementación.

## ContactTrie

Para comenzar el árbol, como todo grafo debemos partir con su unidad básica, un nodo.

In [ ]:
class Node:
    
    def __init__(self):
        self.number = None

Este nodo por ahora sólo contiene el valor del número telefónico que éste podría almacenar. Sin embargo, como lo que queremos es construir un árbol, la idea es que los nodos mantengan una relación entre ellos. Es por esto que, cada nodo va a guardar en un segundo atributo llamado `children` a todos sus nodos hijos. Por otro lado, sabemos que cada nodo va a estar relacionado con una letra, y que la idea es que cada nodo padre no tenga letras repetidas entre sus hijos. Es por esto, que la estructura más adecuada para almacenar a los hijos es un `defaultdict`, cuyas claves sean las letras y para cada una de estas el valor sea un nodo. De esta forma, nos aseguramos que las letras no se podrán repetir, y que cada letra estará asociada a un nodo en particular.

El nodo iría quedando como continúa:

In [ ]:
# Debemos importar el defaultdict (y aprovechamos de importar namedtuple, ya que la 
# necesitaremos más adelante)

from collections import defaultdict, namedtuple


class Node:
    
    def __init__(self):
        self.number = None
        self.children = defaultdict(Node)

Ahora, crearemos la clase `ContactTrie`, la cual será nuestra interfaz. Esta clase actuará como el nodo raíz, y tendrá como hijos a elementos de la clase `Node` (que a la vez tendrán a sus propios hijos), por lo que al igual que en la clase anterior, deberá guardar una relación con ellos. Para esto, contará con el atributo `children` que al igual que en el caso de `Node` será un `defaultdict` para asegurar que no haya repetición de letras y que cada letra esté asociada a un nodo.


In [ ]:
class ContacTrie:

    def __init__(self):
        self.children = defaultdict(Node)

Ahora, comenzaremos a añadir los métodos pedidos a nuestro `ContactTrie`:

En primer lugar, comenzaremos con la funcionalidad de añadir contactos, `add_contact(name, number)`:

> Este método permite agregar contactos entregándole un string con el nombre y un entero con el número del contacto que se desea añadir. Antes de agregar un nuevo contacto se debe verificar que éste sea válido, es decir, que su nombre sea un string que solo contenga letras (ya sean mayúsculas, minúsculas o con tildes) y que no sea vacío; de lo contrario debe imprimir que el nombre ingresado no es válido. Si el string es válido, debe ser transformado a mayúsculas. Además, se debe verificar que el número telefónico ingresado, sea un entero mayor a 0. Si el número entregado no es entero, o es menor o igual a 0, se debe imprimir un mensaje señalando este error. Si ambos argumentos (nombre y número) son válidos, se puede proceder a agregarlos. Si el nombre del contacto ya existe en el ContacTrie, éste se debe sobreescribir, y por ende se modifica el número que ya existía por el nuevo número; de lo contrario, solo de debe añadir el nuevo número.

En primer lugar nos encargaremos de resolver cómo implementar esta funcionalidad, y hacia el final de la ayudantía nos preocuparemos de la validación de los inputs.

Para esto, lo primero que haremos será implementar esta función de agregar hijos para cada nodo, y luego la implementaremos al `ContactTrie`. 


In [ ]:
class Node:
    
    def __init__(self):
        self.number = None
        self.children = defaultdict(Node)

    def _add_contact(self, name, number):
    
            if len(name) > 0:
                # Gracias al defaultdict, podemos obviar el hecho de si existe o no el nodo.
                self.children[name[0]]._add_contact(name[1:], number)
            else:
                if self.number: # Si tiene un número
                    print("Número de contacto actualizado")  
                else:
                    print("Contacto agregado con éxito")
                self.number = number


Al agregar un nuevo contacto a un nodo, se analiza la primera letra del nombre que se entrega como argumento. Si el nodo no tiene esa letra como _key_ de uno de sus nodos hijos, ésta se crea junto a su nuevo nodo hijo correspondiente. Luego esta función toma al nodo hijo cuya _key_ es la primera letra del nombre entregado, y le aplica esta misma función (recursivamente), añadiéndole el nombre **desde la siguiente letra en adelante**. Esto ocurre sucesivamente hasta que se llega a la última letra de la palabra. Cuando ya no hay más letras que agregar, el nodo chequea si tiene un número almacenado en su atributo `number`. Si no tiene, agrega el número e imprime un mensaje que el contacto fue añadido exitosamente, de lo contrario, cambia el número que había previamente guardado por el nuevo número agregado.

Ahora que ya tenemos esta funcionalidad implementada para los nodos, debemos agregarla a `ContactTrie`:

In [ ]:
class ContacTrie:

    def __init__(self):
        self.children = defaultdict(Node)

    def add_contact(self, name, number):
        name = name.upper()
        # Gracias al defaultdict, podemos obviar el hecho de si existe o no el nodo.
        self.children[name[0]]._add_contact(name[1:], number)

Este método en primer lugar toma el nombre a ser añadido y lo convierte a mayúsculas, para luego, haciendo uso de la misma lógica explicada anteriormente, agregar el resto del contacto a aquel nodo hijo que esté asociado a la primera letra del contacto, y así sucesivamente.

In [ ]:
# A continuación se muestra un ejemplo de uso de esta funcionalidad

nuevo_contacttrie=ContacTrie()
nuevo_contacttrie.add_contact("Joaquín", 12345678)
nuevo_contacttrie.add_contact("Tamara", 87654321)
nuevo_contacttrie.add_contact("Joaquín", 12341234)
nuevo_contacttrie.add_contact("Miguel", 45454545)

El siguiente método a implementar, es `get_all_contacts`:

> Este método no recibe argumentos y debe retornar una lista con todos los contactos del ContactTrie. Cada contacto en la lista debe estar contenido en una namedtuple `Contact` con los argumentos `name` y `number`. El orden en que se muestran los contactos es irrelevante.

 Para este método, al igual que el método anterior, crearemos una función en la clase `Node` que nos permita recorrer las ramas de los distintos nodos, y posteriormente haremos uso de esta función para implementar el método en nuestro `ContactTrie`.

In [ ]:
Contact = namedtuple("Contact", ["name","number"])

En primer lugar definimos (fuera de la clase nodo) la namedtuple `Contact`, que nos permitirá almacenar tanto el nombre como el número de cada contacto, para después poder retornarlos en una lista.

In [ ]:
class Node:
    
    def __init__(self):
        self.number = None
        self.children = defaultdict(Node)

    def _add_contact(self, name, number):
    
            if len(name) > 0:
                self.children[name[0]]._add_contact(name[1:], number)
            else:
                if self.number is None:
                    self.number = number
                    print("Contacto agregado con éxito")
                else:
                    self.number = number
                    print("Número de contacto actualizado")  
    
    def _get_all_contacts(self, actual_name, lista):
    
            if self.number:
                lista.append(Contact(actual_name, self.number))
                
            for child in self.children:
                aux = actual_name + child
                new_node = self.children[child]
                new_node._get_all_contacts(aux, lista)


Luego, creamos el método `_get_all_contacts` para la clase `Node`. Este método recibe una palabra (o una letra) y una lista como argumentos. Lo que hace el nodo, es recorrer cada uno de sus hijos, y agregar la letra asociada a cada hijo a la palabra entregada como argumento, así, va avanzando por cada rama, de cada hijo hasta el final. Cuando llega a un nodo con número válido, crea una `Contact` con el nombre del contacto (que se fue construyendo al concatenar todas las letras de la rama desde el nodo inicial) y su número y la añade a la lista.

Posteriormente implementamos este método para la clase `ContactTrie`, como se muestra a continuación:

In [ ]:
class ContacTrie:

    def __init__(self):
        self.children = defaultdict(Node)

    def add_contact(self, name, number):
        name = name.upper()
        self.children[name[0]]._add_contact(name[1:], number)
    
    def get_all_contacts(self):
            contacts = list()
            for child in self.children:
                self.children[child]._get_all_contacts(child, contacts)
            return contacts

In [ ]:
# Para agregar contactos

nuevo_contacttrie = ContacTrie()
nuevo_contacttrie.add_contact("Joaquín", 12345678)
nuevo_contacttrie.add_contact("Tamara", 87654321)
nuevo_contacttrie.add_contact("Gonzalo", 12341234)
nuevo_contacttrie.add_contact("Miguel", 45454545)

# Para visualizar los contactos agregados anteriormente:

lista_contactos = nuevo_contacttrie.get_all_contacts()
print(lista_contactos)
for contacto in lista_contactos:
    print(str(contacto.name)+" "+str(contacto.number))


Este método lo que hace es iterar sobre cada uno de sus hijos, es decir, sobre cada letra que tiene la raíz principal del árbol como hija, y entrega esta letra como argumento, a su nodo hijo correspondiente, para que éste, haciendo uso de la función definida para los nodos, pueda recorrer todas sus ramas hacia abajo e ir agregando los contactos a la lista `contacts`.

Finalmente, implementamos el método `ask_for_contact(name)`:

> Este método debe consultar por un número de un contacto. Para esto se le debe entregar un string con el nombre del contacto pedido. Debe ser transformado a mayúsculas para poder iniciar la búsqueda. Si el contacto no existe se debe imprimir "Contacto Inexistente". Si el contacto existe se debe imprimir el nombre y el número de la siguiente forma: {({nombre\}, \{número\})}''.

A continuación se muestra la función que lo implementa. La idea es utilizar la estructura del árbol para buscar y no abusar del método `get_all_contacts`, ya que el tamaño del árbol puede ser enorme.

In [ ]:
class ContacTrie:

    def __init__(self):
        self.children = defaultdict(Node)

    def add_contact(self, name, number):
        name = name.upper()
        self.children[name[0]]._add_contact(name[1:], number)

    def get_all_contacts(self):
            contacts = list()
            for child in self.children:
                self.children[child]._get_all_contacts(child, contacts)
            return contacts

    def ask_for_contact(self, name):
        
            name = name.upper()
            node = self
    
            initial_name = name
            while name:
    
                if name[0] in node.children:
                    node = node.children[name[0]]
                    name = name[1:]
                else:
                    print("Contacto Inexistente")
                    return
            if node.number:
                print("({}, {})".format(initial_name, node.number))
            else:
                print("Contacto Inexistente")


In [ ]:
# Para agregar contactos


nuevo_contacttrie=ContacTrie()
nuevo_contacttrie.add_contact("Joaquín", 12345678)
nuevo_contacttrie.add_contact("Tamara", 87654321)
nuevo_contacttrie.add_contact("Gonzalo", 12341234)
nuevo_contacttrie.add_contact("Miguel", 45454545)

#Para obtener todos los contactos

lista_contactos=nuevo_contacttrie.get_all_contacts()
for contacto in lista_contactos:
    print(str(contacto.name)+" "+str(contacto.number))

# Para preguntar por el número de Joaquín:

nuevo_contacttrie.ask_for_contact("Joaquín")
nuevo_contacttrie.ask_for_contact("hola")

Este método se utiliza para buscar un contacto. En primer lugar, se recorren los hijos del nodo raíz, verificando que alguno de ellos tenga la primera letra del contacto que se desea buscar. En caso que no se encuentre, la función no retorna nada, pues el contacto era inexistente. Por otro lado, si la letra existía, entonces la función busca la segunda letra del contacto en los nodos _nietos_ del nodo raíz, es decir, en los nodos hijos de aquel nodo hijo que estaba asociado a la primera letra del contacto. De esta forma, va recorriendo el árbol hasta llegar al final del contacto. Si al estar al final de éste, existe un número asociado, entonces retorna el nombre del contacto y su número; de lo contario, no retorna nada.

Como se pueden dar cuenta, en ambas funcionalidades del ContactTrie, cumplimos con la creación de los método, sin embargo, no realizamos la validación de inputs para ninguno de ellos. Sin embargo, esto no es un problema porque podemos crear un decorador que se encargue de esa validación y con ello queda solucionado el problema de los inputs !!!!

A continuación creamos un decorador que podermos utilizar para la validación de inputs tanto de "add_contact" como de "ask_for_contact":

In [ ]:
def check_types(funcion_a_decorar):
    def wrapper(*args, **kwargs):
        if isinstance(args[1], str):
            # A continuación se crean dos funciones, valid_name y valid_number, cuya f
            if len(args[1]) == 0:
                print("String vacío")
                return
            if not args[1].isalpha():
                print("el parámetro debe contener sólo letras")
                return True
        else:
            print("El nombre no es un string")
            return
        if len(args)>2:
            
            if isinstance(args[2], int):
                if not args[2] > 0:
                    print("El número no es mayor que cero")
                    return
            else:
                print("El número no es un entero")
                return
        return funcion_a_decorar(*args, **kwargs)
    return wrapper

Finalmente lo único que debemos hacer es decorar los métodos `add_contact` y `ask_for_contact` de la clase `ContactTrie`, y nuestro programa está listo :) 

In [ ]:
class ContacTrie:

    def __init__(self):
        self.children = defaultdict(Node)
    
    @check_types
    def add_contact(self, name, number):
        name = name.upper()
        self.children[name[0]]._add_contact(name[1:], number)

    def get_all_contacts(self):
            contacts = list()
            for child in self.children:
                self.children[child]._get_all_contacts(child, contacts)
                
            return contacts
    @check_types
    def ask_for_contact(self, name):
        
            name = name.upper()
            node = self
    
            initial_name = name
            while len(name) > 0:
    
                if name[0] in node.children:
                    node = node.children[name[0]]
                    name = name[1:]
                else:
                    print("Contacto Inexistente")
                    return
            if node.number is not None:
                print("({}, {})".format(initial_name, node.number))
            else:
                print("Contacto Inexistente")
            

In [ ]:
# Ahora al intentar ingresar un nombre inválido, el ContactTrie no lo permitirá:

nuevo_contacttrie2=ContacTrie()

nuevo_contacttrie2.add_contact("Miguel", 45454545)

nuevo_contacttrie2.add_contact("Joaqu33n", 12345678)

nuevo_contacttrie2.add_contact("" , 9090909)

nuevo_contacttrie2.add_contact(123,123)


# Aquí se podrá ver sólo el primer contacto pudo ser ingresado, y como ninguno de los siguientes
# tres contactos era válido, entonces el el contacttrie solo registró el primero:

print("")
lista_contactos_2=nuevo_contacttrie2.get_all_contacts()
print("la nueva lista de contactos es: "+str(lista_contactos_2))

#Finalmente, si deseamos buscar un contacto y no ingresamos los parámetros en forma correcta, la búsqueda
#no se realizará:

print("")
nuevo_contacttrie2.ask_for_contact("123")
